In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()


## User Defined Functions

In [3]:
def cubo(n):
    return n * n * n

In [4]:
from pyspark.sql.types import LongType

In [5]:
spark.udf.register("cubo", cubo, LongType())

<function __main__.cubo(n)>

In [6]:
spark.range(1,10).createOrReplaceTempView('df_temp')

In [9]:
spark.sql('select * from df_temp').show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [7]:
spark.sql('select id, cubo(id) as cubo from df_temp').show()

+---+----+
| id|cubo|
+---+----+
|  1|   1|
|  2|   8|
|  3|  27|
|  4|  64|
|  5| 125|
|  6| 216|
|  7| 343|
|  8| 512|
|  9| 729|
+---+----+



In [24]:
def bienvenida(nombre):
  return('Hola {}'.format(nombre))

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [25]:
bienvenida_udf = udf(lambda x: bienvenida(x),StringType())

In [28]:
from pyspark.sql import functions as F

In [29]:
df_nombre = spark.createDataFrame([('Jose',), ('Julia',)], ['nombre'])


In [32]:
df_nombre.select(
  F.col('nombre'),
  bienvenida_udf(F.col('nombre')).alias('nombre hola')
).show()

+------+-----------+
|nombre|nombre hola|
+------+-----------+
|  Jose|  Hola Jose|
| Julia| Hola Julia|
+------+-----------+



In [33]:
@udf(returnType=StringType())
def mayuscula(s):
    return s.upper()

In [35]:
df_nombre.select(
    F.col('nombre'),
    mayuscula(F.col('nombre')).alias('may_nombre')
).show()

+------+----------+
|nombre|may_nombre|
+------+----------+
|  Jose|      JOSE|
| Julia|     JULIA|
+------+----------+



Las UDF de pyspark tienen un rendimiento más lento que las udf en scala ya que requieren movimiento de datos entre la maquina virtual de java y python.  

In [37]:
import pandas as pd

In [38]:
from pyspark.sql.functions import pandas_udf

In [39]:
def cubo_pandas(s: pd.Series) -> pd.Series:
    return s * s * s

In [40]:
cubo_udf = pandas_udf(cubo_pandas, returnType=LongType())

In [41]:
x = pd.Series([1, 2, 3])

print(cubo_pandas(x))

0     1
1     8
2    27
dtype: int64


In [43]:
df = spark.range(5)

df.select(
    F.col('id'),
    cubo_udf(F.col('id')).alias('cubo_pandas')
).show()

+---+-----------+
| id|cubo_pandas|
+---+-----------+
|  0|          0|
|  1|          1|
|  2|          8|
|  3|         27|
|  4|         64|
+---+-----------+



## Window Functions

In [45]:
df_w = spark.read.parquet('/content/funciones_ventana.parquet',inferSchema=True)

In [49]:
df_w.show()

+-------+----+------------+----------+
| nombre|edad|departamento|evaluacion|
+-------+----+------------+----------+
| Lazaro|  45|      letras|        98|
|   Raul|  24|  matemática|        76|
|  Maria|  34|  matemática|        27|
|   Jose|  30|     química|        78|
| Susana|  51|     química|        98|
|   Juan|  44|      letras|        89|
|  Julia|  55|      letras|        92|
|  Kadir|  38|arquitectura|        39|
| Lilian|  23|arquitectura|        94|
|   Rosa|  26|      letras|        91|
|   Aian|  50|  matemática|        73|
|Yaneisy|  29|      letras|        89|
|Enrique|  40|     química|        92|
|    Jon|  25|arquitectura|        78|
|  Luisa|  39|arquitectura|        94|
+-------+----+------------+----------+



In [51]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, desc, rank,dense_rank

In [61]:
df_w.select(
    col('nombre'),
    col('departamento'),
    row_number().over(Window.partitionBy('departamento').orderBy(col('evaluacion').desc())).alias('row_number')
).show()

+-------+------------+----------+
| nombre|departamento|row_number|
+-------+------------+----------+
| Lilian|arquitectura|         1|
|  Luisa|arquitectura|         2|
|    Jon|arquitectura|         3|
|  Kadir|arquitectura|         4|
| Lazaro|      letras|         1|
|  Julia|      letras|         2|
|   Rosa|      letras|         3|
|   Juan|      letras|         4|
|Yaneisy|      letras|         5|
|   Raul|  matemática|         1|
|   Aian|  matemática|         2|
|  Maria|  matemática|         3|
| Susana|     química|         1|
|Enrique|     química|         2|
|   Jose|     química|         3|
+-------+------------+----------+



In [63]:
windowSpec = Window.partitionBy('departamento').orderBy(col('evaluacion').desc())
#

In [69]:
df_w.withColumn('row_number', row_number().over(windowSpec)).filter(col('row_number') == 1).show()

+------+----+------------+----------+----------+
|nombre|edad|departamento|evaluacion|row_number|
+------+----+------------+----------+----------+
|Lilian|  23|arquitectura|        94|         1|
|Lazaro|  45|      letras|        98|         1|
|  Raul|  24|  matemática|        76|         1|
|Susana|  51|     química|        98|         1|
+------+----+------------+----------+----------+



In [71]:
# rank
df_w.withColumn('rank', rank().over(windowSpec)).show()


+-------+----+------------+----------+----+
| nombre|edad|departamento|evaluacion|rank|
+-------+----+------------+----------+----+
| Lilian|  23|arquitectura|        94|   1|
|  Luisa|  39|arquitectura|        94|   1|
|    Jon|  25|arquitectura|        78|   3|
|  Kadir|  38|arquitectura|        39|   4|
| Lazaro|  45|      letras|        98|   1|
|  Julia|  55|      letras|        92|   2|
|   Rosa|  26|      letras|        91|   3|
|   Juan|  44|      letras|        89|   4|
|Yaneisy|  29|      letras|        89|   4|
|   Raul|  24|  matemática|        76|   1|
|   Aian|  50|  matemática|        73|   2|
|  Maria|  34|  matemática|        27|   3|
| Susana|  51|     química|        98|   1|
|Enrique|  40|     química|        92|   2|
|   Jose|  30|     química|        78|   3|
+-------+----+------------+----------+----+



In [73]:
# dense_rank
df_w.withColumn('dense_rank', dense_rank().over(windowSpec)).show()

+-------+----+------------+----------+----------+
| nombre|edad|departamento|evaluacion|dense_rank|
+-------+----+------------+----------+----------+
| Lilian|  23|arquitectura|        94|         1|
|  Luisa|  39|arquitectura|        94|         1|
|    Jon|  25|arquitectura|        78|         2|
|  Kadir|  38|arquitectura|        39|         3|
| Lazaro|  45|      letras|        98|         1|
|  Julia|  55|      letras|        92|         2|
|   Rosa|  26|      letras|        91|         3|
|   Juan|  44|      letras|        89|         4|
|Yaneisy|  29|      letras|        89|         4|
|   Raul|  24|  matemática|        76|         1|
|   Aian|  50|  matemática|        73|         2|
|  Maria|  34|  matemática|        27|         3|
| Susana|  51|     química|        98|         1|
|Enrique|  40|     química|        92|         2|
|   Jose|  30|     química|        78|         3|
+-------+----+------------+----------+----------+



In [75]:
windowSpecAgg = Window.partitionBy('departamento')

In [79]:
from pyspark.sql.functions import min, max, avg

(df_w.withColumn('min', min(col('evaluacion')).over(windowSpecAgg))
.withColumn('max', max(col('evaluacion')).over(windowSpecAgg))
.withColumn('avg', avg(col('evaluacion')).over(windowSpecAgg))
.withColumn('row_number', row_number().over(windowSpec))
 ).show()

+-------+----+------------+----------+---+---+------------------+----------+
| nombre|edad|departamento|evaluacion|min|max|               avg|row_number|
+-------+----+------------+----------+---+---+------------------+----------+
| Lilian|  23|arquitectura|        94| 39| 94|             76.25|         1|
|  Luisa|  39|arquitectura|        94| 39| 94|             76.25|         2|
|    Jon|  25|arquitectura|        78| 39| 94|             76.25|         3|
|  Kadir|  38|arquitectura|        39| 39| 94|             76.25|         4|
| Lazaro|  45|      letras|        98| 89| 98|              91.8|         1|
|  Julia|  55|      letras|        92| 89| 98|              91.8|         2|
|   Rosa|  26|      letras|        91| 89| 98|              91.8|         3|
|   Juan|  44|      letras|        89| 89| 98|              91.8|         4|
|Yaneisy|  29|      letras|        89| 89| 98|              91.8|         5|
|   Raul|  24|  matemática|        76| 27| 76|58.666666666666664|         1|